In [ ]:
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits, ascii
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyvo.dal import TAPService
from pyia import GaiaData

import gala.coordinates as gc

Bigger polygon!

In [ ]:
for r in gc.GD1(phi1=[280, 280, 300, 300]*u.deg, 
                phi2=[-5, 5, 5, -5]*u.deg).transform_to(coord.ICRS):
    print('{0:.3f}, {1:.3f},'.format(r.ra.degree, r.dec.degree))

# for r in gc.GD1(phi1=[340, 340, 360, 360]*u.deg, 
#                 phi2=[-5, 5, 5, -5]*u.deg).transform_to(coord.ICRS):
#     print('{0:.3f}, {1:.3f},'.format(r.ra.degree, r.dec.degree))

```
SELECT *
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > 0 AND bp_rp < 1.75 AND
      CONTAINS(POINT('ICRS', ra, dec), 
               POLYGON('ICRS', 
					   172.933, 46.473,
163.017, 54.242,
197.478, 64.308,
201.885, 54.556)) = 1 
```

In [ ]:
# tap = TAPService("http://tapvizier.u-strasbg.fr/TAPVizieR/tap")

In [ ]:
# query = """SELECT TOP 100 *
# FROM \"I/345/gaia2\"
# WHERE parallax < 1 AND
#       DISTANCE(POINT('ICRS', 34.59, 29.73),
#                POINT('ICRS', ra, dec)) > 87 AND
#       DISTANCE(POINT('ICRS', 34.59, 29.73),
#                POINT('ICRS', ra, dec)) < 93
# """
# # res = tap.search(query, maxrec=100000)
# job = tap.submit_job(query)

In [ ]:
t1 = Table(fits.getdata('../data/gd1-polygon.fits'))
t2 = Table(fits.getdata('../data/gd1-polygon2.fits'))
t3 = Table(fits.getdata('../data/gd1-polygon3.fits'))
t = vstack((t1, t2, t3))

In [ ]:
g = GaiaData(t)
c = coord.SkyCoord(ra=g.ra, dec=g.dec,
                   pm_ra_cosdec=g.pmra, pm_dec=g.pmdec)
gd1_c = c.transform_to(gc.GD1)
# g = g[g.parallax > 0]

In [ ]:
def gd1_dist(phi1):
    # 0, 10
    # -60, 7
    m = (10-7) / (60)
    return (m*phi1.wrap_at(180*u.deg).value + 10) * u.kpc

In [ ]:
gd1_c_dist = gc.GD1(phi1=gd1_c.phi1, phi2=gd1_c.phi2,
                    distance=gd1_dist(gd1_c.phi1),
                    pm_phi1_cosphi2=gd1_c.pm_phi1_cosphi2,
                    pm_phi2=gd1_c.pm_phi2,
                    radial_velocity=[0]*len(gd1_c)*u.km/u.s)

# Correct for reflex motion
v_sun = coord.Galactocentric.galcen_v_sun
observed = gd1_c_dist.transform_to(coord.Galactic)
rep = observed.cartesian.without_differentials()
rep = rep.with_differentials(observed.cartesian.differentials['s'] + v_sun)
# gd1_c_nosunv = coord.Galactic(rep).transform_to(gc.GD1)
gd1_c = coord.Galactic(rep).transform_to(gc.GD1)

In [ ]:
np.isnan(g.pmra).sum()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

ax = axes[0]
ax.plot(g.pmra, g.pmdec, marker=',', linestyle='none', alpha=0.1)

ax.set_xlim(-15, 15.)
ax.set_ylim(-15, 15)
ax.set_xlabel(r'$\mu_\alpha$')
ax.set_ylabel(r'$\mu_\delta$')

ax = axes[1]
ax.plot(gd1_c.pm_phi1_cosphi2, gd1_c.pm_phi2, marker=',', linestyle='none', alpha=0.1)

ax.set_xlim(-30, 30.)
ax.set_ylim(-30, 30)
ax.set_xlabel(r'$\mu_{\phi,1}$')
ax.set_ylabel(r'$\mu_{\phi,2}$')

fig.tight_layout()

In [ ]:
# pm_mask = ((gd1_c.pm_phi1_cosphi2 < -10*u.mas/u.yr) & (gd1_c.pm_phi1_cosphi2 > -25*u.mas/u.yr) & 
#            (gd1_c.pm_phi2 < -2.5*u.mas/u.yr) & (gd1_c.pm_phi2 > -7.5*u.mas/u.yr) & 
#            (g.bp_rp < 1.5*u.mag) & (g.bp_rp > 0*u.mag))
pm_mask = ((gd1_c.pm_phi1_cosphi2 < -5*u.mas/u.yr) & (gd1_c.pm_phi1_cosphi2 > -10*u.mas/u.yr) & 
           (gd1_c.pm_phi2 < 1*u.mas/u.yr) & (gd1_c.pm_phi2 > -2*u.mas/u.yr) & 
           (g.bp_rp < 1.5*u.mag) & (g.bp_rp > 0*u.mag))
pm_mask.sum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

ax.plot(gd1_c.phi1[pm_mask], gd1_c.phi2[pm_mask], marker='.', alpha=0.5, linestyle='none')
# ax.plot(gd1_c.phi1, gd1_c.phi2, marker='.', alpha=0.05, linestyle='none')

In [ ]:
phi2_mask = np.abs(gd1_c.phi2) < 1*u.deg

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

ax = axes[0]
ax.plot(g.pmra[phi2_mask], g.pmdec[phi2_mask], marker=',', linestyle='none', alpha=0.25)

ax.set_xlim(-30, 30.)
ax.set_ylim(-30, 30)
ax.set_xlabel(r'$\mu_\alpha$')
ax.set_ylabel(r'$\mu_\delta$')

ax = axes[1]
ax.plot(gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)[phi2_mask], 
        gd1_c.pm_phi2.to(u.mas/u.yr)[phi2_mask], marker=',', linestyle='none', alpha=0.25)

ax.set_xlim(-30, 30.)
ax.set_ylim(-30, 30)
ax.set_xlabel(r'$\mu_{\phi,1}$')
ax.set_ylabel(r'$\mu_{\phi,2}$')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
better_pm_mask = ((gd1_c.pm_phi1_cosphi2 < -5.5*u.mas/u.yr) & (gd1_c.pm_phi1_cosphi2 > -8.5*u.mas/u.yr) & 
                  (gd1_c.pm_phi2 < 0.1*u.mas/u.yr) & (gd1_c.pm_phi2 > -1.25*u.mas/u.yr) & 
                  (g.bp_rp < 1.5*u.mag) & (g.bp_rp > 0*u.mag))
better_pm_mask.sum()

In [ ]:
color_mask = ((g.phot_g_mean_mag > (5*g.bp_rp + 12.5*u.mag)) | 
              (g.phot_g_mean_mag < (5*g.bp_rp + 9.5*u.mag)))

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(g.bp_rp[better_pm_mask], g.phot_g_mean_mag[better_pm_mask], 
        marker='.', linestyle='none', alpha=0.2)
ax.plot(g.bp_rp[better_pm_mask & color_mask], g.phot_g_mean_mag[better_pm_mask & color_mask], 
        marker='.', linestyle='none', alpha=0.2)

xx = np.linspace(0, 2, 128)
ax.plot(xx, 5*xx + 12.5)
ax.plot(xx, 5*xx + 9.5)

ax.set_ylim(20, 10)
ax.set_xlim(-0.5, 3.)
fig.tight_layout()

In [ ]:
phi2_mask = np.abs(gd1_c.phi2) < 1*u.deg

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

ax = axes[0]
ax.plot(g.pmra[phi2_mask & color_mask], g.pmdec[phi2_mask & color_mask], 
        marker=',', linestyle='none', alpha=0.25)

ax.set_xlim(-30, 30.)
ax.set_ylim(-30, 30)
ax.set_xlabel(r'$\mu_\alpha$')
ax.set_ylabel(r'$\mu_\delta$')

ax = axes[1]
ax.plot(gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)[phi2_mask & color_mask], 
        gd1_c.pm_phi2.to(u.mas/u.yr)[phi2_mask & color_mask], 
        marker='.', linestyle='none', alpha=0.25)

ax.set_xlim(-12, 0.)
ax.set_ylim(-4, 5)
ax.set_xlabel(r'$\mu_{\phi,1}$')
ax.set_ylabel(r'$\mu_{\phi,2}$')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
gd1_tbl = Table.from_pandas(g[color_mask & better_pm_mask].data)
gd1_tbl.write('gd1_tbl.fits')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

ax.plot(gd1_c.phi1[color_mask & better_pm_mask], 
        gd1_c.phi2[color_mask & better_pm_mask], 
        marker='.', alpha=0.5, linestyle='none')

ax.set_xlabel('$\phi_1$ [degree]')
ax.set_ylabel('$\phi_2$ [degree]')

fig.tight_layout()

In [ ]:
bins1 = np.arange(280, 360+0.5, 0.3)
bins2 = np.arange(-5, 5+0.5, 0.3)

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
gal = gd1_c.transform_to(coord.Galactic)

In [ ]:
H, xe, ye = np.histogram2d(gd1_c.phi1[color_mask & better_pm_mask], 
                           gd1_c.phi2[color_mask & better_pm_mask],
                           bins=(bins1, bins2))
H = gaussian_filter1d(H, 1.2)
fig, ax = plt.subplots(1, 1, figsize=(15, 3))
# plt.pcolormesh(xe, ye, H.T)
plt.pcolormesh(xe, ye, np.log(H.T), vmin=-1.5, vmax=0.6)

In [ ]:
# thing  = np.log(H.T).ravel()
# plt.hist(thing[np.isfinite(thing)], bins='auto');

In [ ]:
OMG_mask = ((gd1_c.phi1 > 320*u.deg) & (gd1_c.phi1 < 330*u.deg) & 
            (gd1_c.phi2 > 0.5*u.deg) & (gd1_c.phi2 < 1.7*u.deg))

OMG_stream_mask = ((gd1_c.phi1 > 320*u.deg) & (gd1_c.phi1 < 330*u.deg) & 
                   (gd1_c.phi2 > -0.5*u.deg) & (gd1_c.phi2 < 0.5*u.deg))

OMG_pm_mask = np.sqrt( (gd1_c.pm_phi1_cosphi2 - (-8*u.mas/u.yr))**2 + 
                       (gd1_c.pm_phi2 - (-0.6*u.mas/u.yr))**2 ) < 0.7*u.mas/u.yr

OMG_control_pm_mask = np.sqrt( (gd1_c.pm_phi1_cosphi2 - (-8*u.mas/u.yr))**2 + 
                               (gd1_c.pm_phi2 - (1.6*u.mas/u.yr))**2 ) < 0.7*u.mas/u.yr

(color_mask & OMG_mask).sum()

In [ ]:
(color_mask & OMG_mask).sum()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax = axes[0]
ax.plot(gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)[color_mask & OMG_mask], 
        gd1_c.pm_phi2.to(u.mas/u.yr)[color_mask & OMG_mask], 
        marker='.', linestyle='none', alpha=0.25)

ax.set_xlim(-12, 0.)
ax.set_ylim(-4, 5)
ax.set_xlabel(r'$\mu_{\phi,1}$')
ax.set_ylabel(r'$\mu_{\phi,2}$')

ax.scatter(-8, -0.6, marker='o', facecolor='none', 
           edgecolor='k', linewidth=1, s=1200)

ax = axes[1]
ax.plot(gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)[color_mask & OMG_stream_mask], 
        gd1_c.pm_phi2.to(u.mas/u.yr)[color_mask & OMG_stream_mask], 
        marker='.', linestyle='none', alpha=0.25)

# ax.scatter(-13.2, -3.4, marker='o', facecolor='none', 
#            edgecolor='k', linewidth=1, s=700)
ax.scatter(-7.9, -0.4, marker='o', facecolor='none', 
           edgecolor='k', linewidth=1, s=1500)

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
d_pm = np.sqrt((-13.2 - (-12.9))**2 + .4**2) * u.mas/u.yr
(d_pm * 8*u.kpc).to(u.km/u.s, u.dimensionless_angles())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(g.bp_rp[better_pm_mask], g.phot_g_mean_mag[better_pm_mask], 
        marker='.', linestyle='none', alpha=0.2)

ax.plot(g.bp_rp[color_mask & OMG_mask & OMG_pm_mask], g.phot_g_mean_mag[color_mask & OMG_mask & OMG_pm_mask], 
        marker='o', linestyle='none', alpha=1., color='yellow', markeredgecolor='k', markeredgewidth=1)

ax.plot(g.bp_rp[color_mask & OMG_mask & OMG_control_pm_mask], 
        g.phot_g_mean_mag[color_mask & OMG_mask & OMG_control_pm_mask], 
        marker='o', linestyle='none', alpha=1., color='tab:red', markeredgecolor='k', markeredgewidth=1)

ax.set_ylim(20, 10)
ax.set_xlim(-0.5, 3.)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

ax.quiver(gd1_c.phi1[color_mask & better_pm_mask].value, 
          gd1_c.phi2[color_mask & better_pm_mask].value,
          gd1_c.pm_phi1_cosphi2[color_mask & better_pm_mask].value,
          gd1_c.pm_phi2[color_mask & better_pm_mask].value,
          alpha=0.5, headwidth=4, headlength=5, scale=1E3, width=2E-3,
          rasterized=True)

ax.set_xlabel('$\phi_1$ [degree]')
ax.set_ylabel('$\phi_2$ [degree]')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

ax.quiver(gd1_c.phi1[color_mask & better_pm_mask].value, 
          gd1_c.phi2[color_mask & better_pm_mask].value,
          gd1_c.pm_phi1_cosphi2[color_mask & better_pm_mask].value,
          gd1_c.pm_phi2[color_mask & better_pm_mask].value,
          alpha=0.5, headwidth=4, headlength=5, scale=1E3, width=2E-3,
          rasterized=True)

ax.set_xlabel('$\phi_1$ [degree]')
ax.set_ylabel('$\phi_2$ [degree]')

ax.set_xlim(315, 330)
ax.set_ylim(-2, 2)

# MDM targets

In [ ]:
iso = ascii.read('../data/gd1-isochrone.cmd', header_start=12)

In [ ]:
# control_cut = ((gd1_c.phi1 > 315*u.deg) & (gd1_c.phi1 < 330*u.deg) & 
#                ((gd1_c.phi2 > 1*u.deg) | (gd1_c.phi2 < -1*u.deg)))

control_cut = ((gd1_c.pm_phi1_cosphi2 < -6*u.mas/u.yr) & (gd1_c.pm_phi1_cosphi2 > -8.5*u.mas/u.yr) & 
               (gd1_c.pm_phi2 < 2.5*u.mas/u.yr) & (gd1_c.pm_phi2 > 0*u.mas/u.yr) & 
               (g.bp_rp < 1.5*u.mag) & (g.bp_rp > 0*u.mag))

In [ ]:
mdm_cut = ((g.phot_g_mean_mag < 15*u.mag) & (g.bp_rp > 0.5*u.mag) & 
           (g.phot_g_mean_mag < (5*g.bp_rp + 9.3*u.mag)) &
           (g.phot_g_mean_mag > (-5*g.bp_rp + 22*u.mag)))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 5), sharex=True, sharey=True)

ax = axes[0]
ax.plot(g.bp_rp[better_pm_mask], 
        g.phot_g_mean_mag[better_pm_mask], 
        marker='.', linestyle='none', alpha=0.4)
ax.axvline(0.6)

ax = axes[1]
ax.plot(g.bp_rp[control_cut], 
        g.phot_g_mean_mag[control_cut], 
        marker='.', linestyle='none', alpha=0.4)

# plt.plot(iso['Gaia_BP_DR2Rev'] - iso['Gaia_RP_DR2Rev'],
#          iso['Gaia_G_DR2Rev'] + coord.Distance(gd1_dist(coord.Angle(350*u.deg))).distmod.value)

# xx = np.linspace(0, 2, 128)
# plt.plot(xx, -8*xx + 18.5)

ax.set_xlim(-0.5, 2)
ax.set_ylim(20.7, 9.5)
ax.axvline(0.6)

axes[0].set_title('PM cut')
axes[1].set_title('control')

fig.tight_layout()

In [ ]:
plt.figure(figsize=(6, 6))

plt.plot(g.bp_rp[better_pm_mask], 
         g.phot_g_mean_mag[better_pm_mask], 
         marker='.', linestyle='none', alpha=0.2)

plt.plot(iso['Gaia_BP_DR2Rev'] - iso['Gaia_RP_DR2Rev'],
         iso['Gaia_G_DR2Rev'] + coord.Distance(gd1_dist(coord.Angle(300*u.deg))).distmod.value)
plt.plot(iso['Gaia_BP_DR2Rev'] - iso['Gaia_RP_DR2Rev'],
         iso['Gaia_G_DR2Rev'] + coord.Distance(gd1_dist(coord.Angle(350*u.deg))).distmod.value)

xx = np.linspace(0, 2, 128)
plt.plot(xx, -4.5*xx + 17.8, color='k', marker='')
plt.plot(xx, -4.5*xx + 19.8, color='k', marker='')
plt.plot(xx, 4.5*xx + 10.4, color='k', marker='')
plt.axvline(0.9, color='k', marker='')

plt.xlim(-0.5, 2)
plt.ylim(20.7, 9.5)

In [ ]:
sup = ((g.parallax[better_pm_mask] > 0) & 
       ((g.parallax[better_pm_mask] / g.parallax_error[better_pm_mask]) > 6))

G = g.phot_g_mean_mag[better_pm_mask]
bprp = g.bp_rp[better_pm_mask]

obs_mask = ((G > (-4.5*bprp + 17.8*u.mag)) & 
            (G < (-4.5*bprp + 19.8*u.mag)) & 
            (G < (4.5*bprp + 10.4*u.mag)) &
            (bprp > 0.9*u.mag))
sup &= obs_mask

sup2 = (((g.phot_g_mean_mag[better_pm_mask][sup] - g[better_pm_mask][sup].distmod) < 3.*u.mag) &
        (g.bp_rp[better_pm_mask][sup] > 0.8*u.mag))

len(g.parallax[better_pm_mask][sup][sup2])

In [ ]:
obs_g = g[better_pm_mask][sup][sup2]

In [ ]:
plt.figure(figsize=(6, 6))

plt.plot(g.bp_rp[better_pm_mask], 
         g.phot_g_mean_mag[better_pm_mask], 
         marker='.', linestyle='none', alpha=0.2)

plt.plot(iso['Gaia_BP_DR2Rev'] - iso['Gaia_RP_DR2Rev'],
         iso['Gaia_G_DR2Rev'] + coord.Distance(gd1_dist(coord.Angle(300*u.deg))).distmod.value)
plt.plot(iso['Gaia_BP_DR2Rev'] - iso['Gaia_RP_DR2Rev'],
         iso['Gaia_G_DR2Rev'] + coord.Distance(gd1_dist(coord.Angle(350*u.deg))).distmod.value)

plt.plot(obs_g.bp_rp, 
         obs_g.phot_g_mean_mag, 
         marker='o', linestyle='none', alpha=0.75)

# xx = np.linspace(0, 2, 128)
# plt.plot(xx, -4.5*xx + 17.8, color='k', marker='')
# plt.plot(xx, -4.5*xx + 19.8, color='k', marker='')
# plt.plot(xx, 4.5*xx + 10.4, color='k', marker='')
# plt.axvline(0.9, color='k', marker='')

plt.xlim(-0.5, 2)
plt.ylim(20.7, 9.5)

In [ ]:
gd1_tbl = Table.from_pandas(obs_g.data)
gd1_tbl = gd1_tbl['ra', 'dec', 'phot_g_mean_mag']
gd1_tbl = gd1_tbl[np.argsort(gd1_tbl['ra'])]
gd1_tbl['name'] = ['GD1_{0:02d}'.format(i) for i in range(len(gd1_tbl))]
gd1_tbl = gd1_tbl['name', 'ra', 'dec', 'phot_g_mean_mag']

ra = coord.Angle(gd1_tbl['ra'] * u.deg)
dec = coord.Angle(gd1_tbl['dec'] * u.deg)
aaah = coord.SkyCoord(ra=ra, dec=dec)
gd1_tbl['ra_str'] = ra.to_string(unit=u.hourangle, sep=' ', precision=2)
gd1_tbl['dec_str'] = dec.to_string(unit=u.degree, sep=' ', precision=2)

gd1_tbl = gd1_tbl['name', 'ra_str', 'dec_str', 'phot_g_mean_mag']
gd1_tbl.write('/Users/adrian/projects/mdm-gaia/gd1.txt', format='ascii.basic')

In [ ]:
from astropy.time import Time
from astroplan import FixedTarget, Observer, is_observable, AirmassConstraint, AtNightConstraint

In [ ]:
observer = Observer(location=coord.EarthLocation.of_site('MDM'))

In [ ]:
time_range = Time(['2018-04-25', '2018-04-29'])
constraints = [AirmassConstraint(max=2.5, boolean_constraint=False),
               AtNightConstraint.twilight_civil()]

In [ ]:
targets = [FixedTarget(cc) for cc in aaah]
is_observable(constraints, observer, targets, time_range=time_range)